In [15]:
#Instalation of required packages and libraries for the project

!pip install selenium
!pip install python-whois
!pip install tqdm
!pip install requests-html
#pip install nest-asyncio
import builtwith
import whois
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
from time import sleep
import requests
import random
import time
import pandas as pd

In [16]:
#Previous evaluation, we will inspect Robots.txt, the technology used to build the website and find out the website owner
#i'm using 'benestaranimal site' from 'Ajuntament de Barcelona', this site is both in spanish and catalan this is defined by
#the "ca" or "es" after the "/"at the end of the site. To evaluate Robots.txt i will have to use the roo url  meaning i have to 
#reach https://ajuntament.barcelona.cat/benestaranimal/robots.txt

print ("technology used to develope the website i'm gonna use")
print(builtwith.builtwith('https://ajuntament.barcelona.cat/benestaranimal/ca'))
print ("owner of the website")
print(whois.whois('https://ajuntament.barcelona.cat/benestaranimal/ca'))
print('robots.txt detail')
print(requests.get('https://ajuntament.barcelona.cat/benestaranimal/robots.txt', data=None).text)


technology used to develope the website i'm gonna use
{'web-servers': ['Nginx'], 'cms': ['Drupal'], 'programming-languages': ['PHP'], 'maps': ['Google Maps'], 'web-frameworks': ['Materialize CSS', 'Twitter Bootstrap'], 'javascript-frameworks': ['jQuery', 'jQuery UI'], 'widgets': ['OWL Carousel'], 'photo-galleries': ['jQuery'], 'video-players': ['YouTube']}
owner of the website
{
  "domain_name": "barcelona.cat",
  "registrar": "Entorno Digital",
  "whois_server": null,
  "referral_url": null,
  "updated_date": "2020-01-26 16:47:18.267000",
  "creation_date": "2006-02-22 17:04:03.390000",
  "expiration_date": "2021-02-22 17:04:03.390000",
  "name_servers": [
    "ns2.bcn.cat",
    "ns1.bcn.cat",
    "ns3bcn.entorno.es"
  ],
  "status": "ok https://icann.org/epp#ok",
  "emails": "abuse@entorno.es",
  "dnssec": "unsigned",
  "name": null,
  "org": "Ajuntament de Barcelona",
  "address": null,
  "city": null,
  "state": "Barcelona",
  "zipcode": null,
  "country": "ES"
}
robots.txt detail


In [17]:
# Url that i'm gonna reach:

url="https://ajuntament.barcelona.cat/benestaranimal/ca"

driver=webdriver.Chrome()

driver_setting=webdriver.ChromeOptions()
driver_setting.add_argument("user-agent=Study_purpose_crawler")

driver.get(url)

#i'm putting some delay between clicking buttons just to make sure the page is already loaded.

time.sleep(5)

driver.find_element_by_xpath('//*[@id="main-menu-ul"]/li[4]/span').click()

time.sleep(2)

driver.find_element_by_xpath('//*[@id="main-menu-ul"]/li[4]/div/ul/li[2]/ul/li[2]/ul/li[3]/a').click()


In [18]:
# get links from each pet's site

pets_links=[]
condition=True
while condition:
    pets= driver.find_elements_by_css_selector('.views-row')
    time.sleep(10)
    for pet  in pets:
        pets1=pet.find_element_by_class_name('field-content')
        pet2=pets1.find_element_by_tag_name('a')
        pets_links.append(pet2.get_property('href'))
    try:
                
        driver.find_element_by_xpath("//a[contains(text(),'›')]").click()
        time.sleep(10)
    except:
        condition=False

# check with len function if we have got more links.

len(pets_links)    

In [19]:
#Data from each pet on the refugee

alldata=[]

for pet_link in tqdm(pets_links):
    driver.get(pet_link)
    time.sleep(random.uniform(4.0,5.0))
    
    name= driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[1]/h2').text
    animal_type= driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[1]').text
    race= driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[2]').text
    cross_race=driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[last()-4]').text
    sex= driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[last()-3]').text
    age= driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[last()-2]').text
    weight= driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[last()-1]').text
    id_animal=driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[last()]').text
    link_picture=driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[1]/div/div[2]/div/img').get_attribute('src')
    data= {'name':name,'Type':animal_type,'race':race,'cross_race':cross_race,'sex':sex,'age':age,'weight':weight,'id':id_animal,'link_picture':link_picture,'link_pet_site':pet_link}
    alldata.append(data)
    
driver.quit()

100%|████████████████████████████████████████████████████████████████████████████████| 181/181 [16:01<00:00,  5.31s/it]


In [20]:
#convert the data on a dataframe using pandas:
df=pd.DataFrame(alldata)

#Save as a csv:
df.to_csv (r'C:\Users\mila_\Documents\Master ciencia de dades\Tipología y ciclo de vida de los datos\PRAC 1\pets_in_shelter.csv', index = False, header=True)
print(df)

       name Type                       race cross_race      sex           age  \
0     ROCKY  Gos                     Creuat     Creuat   Mascle         Array   
1       REX  Gos               Bull Terrier     Creuat   Mascle         Array   
2     CAMEL  Gos  Pit Bull Terrirer America     Creuat   Mascle         Array   
3     ARITZ  Gos  Pit Bull Terrirer America     Creuat   Mascle  Mitjana edat   
4       BOY  Gos                     Creuat     Creuat   Mascle         Array   
..      ...  ...                        ...        ...      ...           ...   
176    TICO  Gos         American Stafordsh              Mascle          Jove   
177   KAREN  Gat                    Europeu             Femella          Jove   
178  PEPONA  Gat                    Europeu     Creuat  Femella  Mitjana edat   
179   ROCKY  Gos                     Creuat     Creuat   Mascle          Jove   
180   SULLY  Gos               Gos D 'Atura     Creuat   Mascle         Array   

          weight        id 